In [2]:
!pip install google-play-scraper pandas


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 2.3 MB/s eta 0:00:00


In [2]:
!pip install sastrawi


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 4.1 MB/s eta 0:00:00


In [2]:
pip install gensim


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 8.6 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.14.1
    Uninstalling scipy-1.14.1:
      Successfully uninstalled scipy-1.14.1


In [1]:
import pandas as pd
import numpy as np
import re
import nltk
import gensim
import pickle
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from google_play_scraper import reviews
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from gensim.models import Word2Vec
import joblib

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

## Scraping Data

In [2]:
from google_play_scraper import reviews, Sort
import pandas as pd

# Scrape review dari aplikasi Gojek
app_id = "com.gojek.app"
count = 10000

result, _ = reviews(
    app_id,
    lang='id',  # Bahasa Indonesia
    country='id',  # Indonesia
    sort=Sort.NEWEST,  # Urutkan berdasarkan yang terbaru
    count=count
)

# Konversi hasil scraping ke DataFrame
reviews_df = pd.DataFrame(result)

# Simpan ke CSV agar bisa digunakan untuk analisis lebih lanjut
reviews_df.to_csv("gojek_reviews.csv", index=False)

In [3]:
reviews_df = pd.read_csv("gojek_reviews.csv")
reviews_df.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,be344f8b-7bf2-4ede-8fd6-a3156c608824,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,terus tingkatkan lagi dan berikan inovasi dan ...,5,0,NaN,2025-03-24 16:42:24,NaN,NaN,NaN
1,a2a40ba9-3f43-4cdc-a848-d16d2d99cea3,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,sangat membantu,5,0,5.15.1,2025-03-24 16:31:35,NaN,NaN,5.15.1
2,7b1ac432-3d60-40e8-b0d2-44250db4f4eb,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,numeroo unoo,5,0,5.14.2,2025-03-24 16:26:48,NaN,NaN,5.14.2
3,1a455d7f-e5d4-4bc5-af1a-6196f6a945e1,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,sangat membantu,5,0,4.93.1,2025-03-24 16:14:26,NaN,NaN,4.93.1
4,dd84b72a-fcfd-460d-b253-cf2211a67071,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,mantap,5,0,NaN,2025-03-24 16:05:39,NaN,NaN,NaN


## Preprocessing Teks & Labeling

In [8]:
process_df = reviews_df.copy()

In [9]:
# Pastikan stopwords bahasa Indonesia sudah ada
nltk.download('stopwords')
stopwords_indonesia = set(stopwords.words('indonesian'))

# Inisialisasi stemmer dari Sastrawi
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Pilih hanya kolom yang diperlukan
process_df = reviews_df[['content', 'score']].dropna()
process_df.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,content,score
0,terus tingkatkan lagi dan berikan inovasi dan ...,5
1,sangat membantu,5
2,numeroo unoo,5
3,sangat membantu,5
4,mantap,5


In [11]:
# cleaning dan preprocessing teks
def clean_text(text):
    text = text.lower()  # Lowercasing
    text = re.sub(r'http\S+|www.\S+', '', text)  #
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = text.strip()
    return text

def preprocess_text(text):
    text = clean_text(text)
    words = text.split()  # Tokenisasi manual dengan split
    words = [word for word in words if word not in stopwords_indonesia]
    words = [stemmer.stem(word) for word in words]
    return " ".join(words)

process_df['cleaned_content'] = process_df['content'].astype(str).apply(preprocess_text)

process_df.head(20)

,content,score,cleaned_content
0,terus tingkatkan lagi dan berikan inovasi dan ...,5,tingkat inovasi promo yg tarik guna setia goje...
1,sangat membantu,5,bantu
2,numeroo unoo,5,numeroo unoo
3,sangat membantu,5,bantu
4,mantap,5,mantap
5,senang membantu,5,senang bantu
6,kenapa sekarang susah sekali masuk,5,susah masuk
7,"terlalu tolol, udh tau gaada driver, malah mek...",1,tolol udh tau gaada driver mekanisme batalin p...
8,ok,5,ok
9,oke God,5,oke god


In [19]:
# label sentimen berdasarkan score
def label_sentiment(score):
    if score >= 4:
        return 2  # Positif
    elif score == 3:
        return 1  # Netral
    else:
        return 0  # Negatif

process_df['sentiment'] = process_df['score'].apply(label_sentiment)
process_df.head()

,content,score,cleaned_content,sentiment
0,terus tingkatkan lagi dan berikan inovasi dan ...,5,tingkat inovasi promo yg tarik guna setia goje...,2
1,sangat membantu,5,bantu,2
2,numeroo unoo,5,numeroo unoo,2
3,sangat membantu,5,bantu,2
4,mantap,5,mantap,2


In [21]:
# Pilih kolom yang diperlukan
cleaned_df = process_df[['cleaned_content', 'sentiment']].copy()
cleaned_df.head()

,cleaned_content,sentiment
0,tingkat inovasi promo yg tarik guna setia goje...,2
1,bantu,2
2,numeroo unoo,2
3,bantu,2
4,mantap,2


In [22]:
cleaned_df.to_csv("gojek_reviews_cleaned.csv", index=False)

*note : saya menyimpan data yang sudah cleaned hanya unutk memudahkan proses pengerjaan di hari esok karena project ini tidak saya selesaikan in one sitting*

## Ekstraksi Fitur


In [2]:
# Load data
clean_df = pd.read_csv("gojek_reviews_cleaned.csv")

In [3]:
clean_df.head()

,cleaned_content,sentiment
0,tingkat inovasi promo yg tarik guna setia goje...,2
1,bantu,2
2,numeroo unoo,2
3,bantu,2
4,mantap,2


In [4]:
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   cleaned_content  9731 non-null   object
 1   sentiment        10000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 156.4+ KB


In [5]:
clean_df['cleaned_content'] = clean_df['cleaned_content'].fillna("")


**TF-IDF**

In [22]:
import pickle

# Pisahkan data train dan test
X_train, X_test, y_train, y_test = train_test_split(clean_df['cleaned_content'], clean_df['sentiment'], test_size=0.2, random_state=42)

# Inisialisasi TF-IDF Vectorizer
tfidf = TfidfVectorizer(max_features=5000)  # Maksimal 5000 fitur
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

with open("tfidf_vectorizer.pkl", "wb") as file:
    pickle.dump(tfidf, file)

**Word2Vec**

In [8]:
# Tokenisasi untuk Word2Vec
sentences = [row.split() for row in clean_df['cleaned_content']]
word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=2, workers=4)
word2vec_model.save("word2vec_model.bin")

# Konversi kalimat ke vektor rata-rata dari Word2Vec
def sentence_to_vector(sentence, model, vector_size=100):
    words = sentence.split()
    word_vectors = [model.wv[word] for word in words if word in model.wv]
    return np.mean(word_vectors, axis=0) if word_vectors else np.zeros(vector_size)

X_word2vec = np.array([sentence_to_vector(text, word2vec_model) for text in clean_df['cleaned_content']])
y = clean_df['sentiment']

# Split data
X_train_w2v, X_test_w2v, y_train_w2v, y_test_w2v = train_test_split(X_word2vec, y, test_size=0.3, random_state=42)


# Model

**Model 1 - TF-IDF + SVM**

In [9]:
# Inisialisasi model SVM
svm_model = SVC(kernel='linear')
svm_model.fit(X_train_tfidf, y_train)

# Prediksi
y_pred_svm = svm_model.predict(X_test_tfidf)

# Evaluasi
print("Accuracy (TF-IDF + SVM):", accuracy_score(y_test, y_pred_svm))
print(classification_report(y_test, y_pred_svm))

# Simpan model
joblib.dump(svm_model, "tfidf_svm_model.pkl")


Accuracy (TF-IDF + SVM): 0.88
              precision    recall  f1-score   support

           0       0.78      0.84      0.81       540
           1       0.00      0.00      0.00        77
           2       0.92      0.94      0.93      1383

    accuracy                           0.88      2000
   macro avg       0.57      0.60      0.58      2000
weighted avg       0.85      0.88      0.86      2000



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


['tfidf_svm_model.pkl']

**Model 2 - TF-IDF + Random Forest**

In [10]:
# Inisialisasi model Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_tfidf, y_train)

# Prediksi
y_pred_rf = rf_model.predict(X_test_tfidf)

# Evaluasi
print("Accuracy (TF-IDF + Random Forest):", accuracy_score(y_test, y_pred_rf))
print(classification_report(y_test, y_pred_rf))

# Simpan model
joblib.dump(rf_model, "tfidf_rf_model.pkl")


Accuracy (TF-IDF + Random Forest): 0.8735
              precision    recall  f1-score   support

           0       0.75      0.86      0.80       540
           1       0.50      0.03      0.05        77
           2       0.93      0.92      0.93      1383

    accuracy                           0.87      2000
   macro avg       0.73      0.60      0.59      2000
weighted avg       0.87      0.87      0.86      2000



['tfidf_rf_model.pkl']

**Model 3 - Word2Vec + LSTM**

In [20]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.regularizers import l2


# Konversi label menjadi one-hot encoding
y_train_onehot = to_categorical(y_train, num_classes=3)
y_test_onehot = to_categorical(y_test, num_classes=3)

# Tokenisasi dengan Keras
tokenizer = Tokenizer()
tokenizer.fit_on_texts(clean_df['cleaned_content'])

X_train_seq = pad_sequences(tokenizer.texts_to_sequences(X_train), maxlen=50)
X_test_seq = pad_sequences(tokenizer.texts_to_sequences(X_test), maxlen=50)

# Model LSTM
model_lstm = Sequential([
    Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100, input_length=50),
    LSTM(64, return_sequences=True),
    LSTM(32),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax', kernel_regularizer=l2(0.5))
])

# Compile model (gunakan categorical_crossentropy)
model_lstm.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Training
model_lstm.fit(X_train_seq, y_train_onehot, epochs=5, batch_size=32, validation_data=(X_test_seq, y_test_onehot))

# Evaluasi
loss, accuracy = model_lstm.evaluate(X_test_seq, y_test_onehot)
print("Accuracy (Word2Vec + LSTM):", accuracy)

# Simpan model
model_lstm.save("word2vec_lstm_model.h5")


Epoch 1/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 22s 61ms/step - accuracy: 0.7511 - loss: 2.8895 - val_accuracy: 0.8655 - val_loss: 0.8968
Epoch 2/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 20s 59ms/step - accuracy: 0.8923 - loss: 0.6741 - val_accuracy: 0.8790 - val_loss: 0.4326
Epoch 3/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 15s 60ms/step - accuracy: 0.9264 - loss: 0.3308 - val_accuracy: 0.8790 - val_loss: 0.3926
Epoch 4/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 20s 57ms/step - accuracy: 0.9392 - loss: 0.2577 - val_accuracy: 0.8760 - val_loss: 0.4203
Epoch 5/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 20s 55ms/step - accuracy: 0.9465 - loss: 0.2316 - val_accuracy: 0.8765 - val_loss: 0.4146
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.8795 - loss: 0.4104


Accuracy (Word2Vec + LSTM): 0.8765000104904175


In [30]:
import pickle
import joblib
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.preprocessing.text import Tokenizer

# 1️⃣ Muat kembali model & vectorizer
with open("tfidf_vectorizer.pkl", "rb") as file:
    tfidf_vectorizer = pickle.load(file)

svm_model = joblib.load("tfidf_svm_model.pkl")
rf_model = joblib.load("tfidf_rf_model.pkl")
word2vec_model = Word2Vec.load("word2vec_model.bin")
lstm_model = load_model("word2vec_lstm_model.h5", compile=False)


In [31]:
# 2️⃣ Fungsi untuk konversi teks ke format vektor
def preprocess_text_tfidf(text):
    return tfidf_vectorizer.transform([text])

def preprocess_text_word2vec(text, model, vector_size=100):
    words = text.split()
    word_vectors = [model.wv[word] for word in words if word in model.wv]
    return np.mean(word_vectors, axis=0) if word_vectors else np.zeros(vector_size)

def preprocess_text_lstm(text, tokenizer, max_len=50):
    seq = tokenizer.texts_to_sequences([text])
    return pad_sequences(seq, maxlen=max_len)

# Tokenizer untuk LSTM
tokenizer = Tokenizer()
tokenizer.fit_on_texts(clean_df['cleaned_content'])  # Gunakan data sebelumnya

In [32]:
# 3️⃣ Fungsi untuk melakukan prediksi
def predict_sentiment(text):
    # TF-IDF + SVM
    tfidf_vector = preprocess_text_tfidf(text)
    pred_svm = svm_model.predict(tfidf_vector)[0]

    # TF-IDF + Random Forest
    pred_rf = rf_model.predict(tfidf_vector)[0]

    # Word2Vec + LSTM
    word2vec_vector = np.array([preprocess_text_word2vec(text, word2vec_model)])
    lstm_input = preprocess_text_lstm(text, tokenizer)
    pred_lstm = np.argmax(lstm_model.predict(lstm_input), axis=1)[0]

    return {
        "TF-IDF + SVM": pred_svm,
        "TF-IDF + Random Forest": pred_rf,
        "Word2Vec + LSTM": pred_lstm
    }

In [36]:
# 4️⃣ Contoh Prediksi
text_sample = "aplikasi ini biasa saja "
predictions = predict_sentiment(text_sample)
print(predictions)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
{'TF-IDF + SVM': 2, 'TF-IDF + Random Forest': 2, 'Word2Vec + LSTM': 0}
